In [1]:
import instagrapi
from instagrapi import Client

from dotenv import load_dotenv

import pandas as pd
import numpy as np
import re

from datetime import date, datetime, timedelta
import pytz
from tqdm import tqdm
import time

timezone_utc = pytz.timezone('UTC')
timezone_indo = pytz.timezone('Asia/Jakarta')
load_dotenv()

True

In [2]:
# get username and password from .env file
USERNAME = os.getenv("ACCOUNT_USERNAME")
PASSWORD = os.getenv("ACCOUNT_PASSWORD")

In [4]:
# login to instagram
cl = Client()
cl.login("kazuyatakeshi1", "bimoli216")
# cl.login(USERNAME, PASSWORD)

Status 429: Too many requests
Ignore 429: Continue login


True

In [5]:
profile = cl.user_id_from_username("sugab.3gp")

In [6]:
print(profile)

2248835396


Behavioral data: This includes user interactions, such as likes, comments, shares, and follows, which can reveal interests, preferences, and social networks.

Note: 
- Akun diblokir jika melakukan request terlalu banyak. Jika terjadi error, tunggu beberapa saat dan coba lagi.
- Terkadang terjadi error pada saat mengambil data, karena terlalu banyak request yang dilakukan. Untuk mengatasinya, kita bisa menambahkan delay pada setiap request yang dilakukan. Untuk menambahkan delay, kita bisa menggunakan fungsi time.sleep().

# Scrape Users Profile
This code will scrape users profile data such as:
- 'taken_at_timestamp' (datetime when the profile was scraped)
- 'id_user'
- 'username'
- 'fullname'
- 'is_private'
- 'is_verified'
- 'follower_count'
- 'following_count'
- 'media_count'
- 'like_count' (sum of last 10 post)
- 'comment_count' (sum of last 10 post)
- 'last_postingan' (last post date)
- 'last_postingan_interval' (last_postingan - taken_at_timestamp)
- 'last_postingan_id' (last post id)
- 'last_story' (last story date)
- 'last_story_interval' (last_story - taken_at_timestamp)
- 'last_story_id' (last story id)

In [35]:
# init read users from csv
# insert all users you want to scrape data in the users.csv file
users_list = pd.read_csv("data/users.csv")["users"].tolist()

## Function to scrape user profile

In [36]:

def get_user_info(target_user):
    now = datetime.now(timezone_indo)
    like = 0
    comment = 0
    last_post = None
    last_post_interval = None
    last_story = None
    last_story_interval = None
    
    print("Scraping data from user: ", target_user)
    
    
    # get user id from username
    user_id = cl.user_id_from_username(target_user)
    user_inf = cl.user_info(user_id)
    
    print("Getting media data...")
    print("now: ", now)
    
    # get all media from user (tanpa delay, pasti error keblock akunnya)
    # aku ganti menjadi 10 post terakhir aja
    user_media = cl.user_medias(user_id, amount=10)
    # cek if post exist
    if user_media:
        datetime_list = []
        
        # get like count, comment count, and latest datetime from all media
        for media in user_media:
            like += media.like_count
            comment += media.comment_count
            datetime_list.append(media.taken_at)
        
        latest_datetime = max(datetime_list)
        latest_index = datetime_list.index(latest_datetime)
        
        last_post = str(user_media[latest_index].taken_at)
        last_post_obj = datetime.strptime(last_post, '%Y-%m-%d %H:%M:%S%z')
        # print("last post: ", last_post_obj)
        
        # change datetime to indonesia timezone
        last_post_obj = last_post_obj.astimezone(timezone_indo)
        last_post = last_post_obj

        # count interval
        last_post_interval = (now - last_post_obj).days
        # print("last post interval: ", last_post_interval)
    
    
    # get last story from user
    # ngambil 25 stori teratas
    user_story = cl.user_stories(user_id, amount=25)
    # cek if story exist
    if user_story:
        # stori terakhir yang dibuat
        user_story = user_story[-1]
        
        # get current datetime of story
        last_story = str(user_story.taken_at)
        last_story_obj = datetime.strptime(last_story, '%Y-%m-%d %H:%M:%S%z')
        # print("last story: ", last_story_obj)
        
        # change datetime to indonesia timezone
        last_story_obj = last_story_obj.astimezone(timezone_indo)
        last_story = last_story_obj
        
        # count interval
        last_story_interval = (now - last_story_obj).seconds // 3600
        # print("last story interval: ", last_story_interval)
    
    # append all data to profile_list
    profile_list = ({
            'id_user':user_inf.pk, 
            'username':user_inf.username, 
            'fullname':user_inf.full_name, 
            'is_private':user_inf.is_private,  
            'is_verified':user_inf.is_verified,  
            'follower_count': user_inf.follower_count, 
            'following_count': user_inf.following_count,
            'media_count':user_inf.media_count,
            'like_count(last10post)': like,
            'comment_count(last10post)': comment,
            'last_post': last_post,
            'last_post_interval(days)': last_post_interval,
            'last_story': last_story,
            'last_story_interval(hours)': last_story_interval,
            })
    
    print("Scraping finish!")
    return profile_list

## Do scraping user profile

Note: jika ada data yang tidak terambil, maka relogin dan coba lagi. Dan kalau bisa sekali ngambil 20-25 data aja dulu.

In [ ]:
users_profile = []
i = 0

for user in users_list:
    i += 1
    print("Iteration: ", i)
    # get user info
    user_info = get_user_info(user)
    # append user info to user_infos
    users_profile.append(user_info)
    time.sleep(5)
    print()

In [26]:
df_users_profile = pd.DataFrame(users_profile)
df_users_profile

,id_user,username,fullname,biography,is_private,is_verified,follower_count,following_count,media_count,like_count(last10post),comment_count(last10post),last_post,last_post_interval(days),last_story,last_story_interval(hours)
0,1570222727,retnowynti,Retno Wijayanti,,True,False,936,621,0,0,0,NaT,NaN,None,None
1,3654850700,fathonizkr,,u dunno me,False,False,876,780,9,-9,55,2022-11-21 22:18:29+07:00,94.0,None,None
2,2271037529,smrmendem,Dimas Adi,A long way up to the summit.,False,False,118,80,4,32,8,2023-01-13 19:13:28+07:00,41.0,None,None
3,5371286272,hafifarhans,De_Farhana,🥈I want to be useful with all of them aamiin🥺\...,False,False,407,639,33,859,91,2022-01-19 12:18:28+07:00,401.0,None,None
4,7737040222,nandi_ir,Nandi Irawan,,True,False,228,362,3,0,0,NaT,NaN,None,None
5,3101437999,mochilhamsyah_,heuh hayoh,,True,False,566,625,3,0,0,NaT,NaN,None,None


In [27]:
# check if users_profile.csv exist
if os.path.exists("data/users_profile.csv"):
    print("users_profile.csv exist")

    # open last users_profile.csv
    df_last_users_profile = pd.read_csv("data/users_profile.csv")

    # append new data to last data
    df_last_users_profile = df_last_users_profile.concat(df_users_profile, ignore_index=True)
    # df_last_users_profile = df_last_users_profile.append(df_users_profile, ignore_index=True)
    
else:
    print("users_profile.csv doesn't exist")
    
    # create new users_profile.csv
    df_last_users_profile = df_users_profile

users_profile.csv exist


/tmp/ipykernel_3194/2733630993.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_last_users_profile = df_last_users_profile.append(df_users_profile, ignore_index=True)


In [28]:
df_last_users_profile.shape

(56, 15)

In [29]:
df_last_users_profile.tail(2)

,id_user,username,fullname,biography,is_private,is_verified,follower_count,following_count,media_count,like_count(last10post),comment_count(last10post),last_post,last_post_interval(days),last_story,last_story_interval(hours)
54,7737040222,nandi_ir,Nandi Irawan,,True,False,228,362,3,0,0,NaT,NaN,None,None
55,3101437999,mochilhamsyah_,heuh hayoh,,True,False,566,625,3,0,0,NaT,NaN,None,None


In [30]:
df_last_users_profile.to_csv("data/users_profile.csv", index=False)

In [290]:
cl.logout()

True

In [41]:
test = pd.read_csv("data/users_profile.csv")
test.tail()

,id_user,username,fullname,biography,is_private,is_verified,follower_count,following_count,media_count,like_count(last10post),comment_count(last10post),last_post,last_post_interval(days),last_story,last_story_interval(hours)
51,3654850700,fathonizkr,NaN,u dunno me,False,False,876,780,9,-9,55,2022-11-21 22:18:29+07:00,94.0,NaN,NaN
52,2271037529,smrmendem,Dimas Adi,A long way up to the summit.,False,False,118,80,4,32,8,2023-01-13 19:13:28+07:00,41.0,NaN,NaN
53,5371286272,hafifarhans,De_Farhana,🥈I want to be useful with all of them aamiin🥺\...,False,False,407,639,33,859,91,2022-01-19 12:18:28+07:00,401.0,NaN,NaN
54,7737040222,nandi_ir,Nandi Irawan,NaN,True,False,228,362,3,0,0,NaN,NaN,NaN,NaN
55,3101437999,mochilhamsyah_,heuh hayoh,NaN,True,False,566,625,3,0,0,NaN,NaN,NaN,NaN


---

# Scrape Users Following

In [ ]:
# init read users from csv
# insert all users you want to scrape data in the users.csv file
users_list = pd.read_csv("data/users.csv")["users"].tolist()

In [ ]:
users_following = []

# loop through all users
for user in tqdm(users_list):
    following_list = []
    # followers_list = []
    
    # get user information
    user_id = cl.user_id_from_username(user)
    user_info = cl.user_info(user_id)
    follower_count = user_info.follower_count
    following = cl.user_following(user_id)
    
    # get all users following into a list
    for key, val in following.items():
        following_list.append(val.username)
        
    # convert list to string
    following = re.sub(r'[\[\]\']', '', str(following_list))    
    
    # append all data to users_following list
    users_following.append({
        'userId': user_id,
        'username':user,
        'following_count': len(following_list),
        'followers_count': follower_count,
        'following_list': following,
        # # 'followers_list': followers_list
                                })

In [ ]:
# convert list to dataframe
df_following = pd.DataFrame(users_following)
df_following

In [ ]:
# save dataframe to csv
# df_following.to_csv("data/users_following.csv", index=False)

---

# Scrape User Comments on All Target User Posts

In [ ]:
# !!! insert target user here
target_user = "sugab.3gp"

# get user media from target user
medias = cl.user_medias(cl.user_id_from_username(target_user), amount=5)

In [ ]:
target_user_comment = []

# loop through all media
for media in tqdm(medias):
    print("Media ID: ", media.pk)
        
    # get all comments from media
    media_comments = cl.media_comments(media.pk, amount=500)
    # try:
    # except instagrapi.exceptions.LoginRequired:
    #     cl = login(USERNAME, PASSWORD)
    #     media_comments = cl.media_comments(media.pk, amount=500)
    
    # loop through all comments
    for comment in tqdm(media_comments):
        
        is_user = False
        
        # check if comment is from target user
        if comment.user.username == target_user:
            is_user = True
        
        # append all data to target_user_comment list
        target_user_comment.append({
            'created_at': comment.created_at_utc,
            'media_id': media.pk,
            'user_id': comment.user.pk,
            'is_user_post': is_user,
            'username': comment.user.username,
            'has_like': comment.has_liked,
            'like_count': comment.like_count,
            'comment_id': comment.pk,
            'comment': comment.text,
            })

In [ ]:
# convert list to dataframe
df_target_user_comment = pd.DataFrame(target_user_comment)
df_target_user_comment.head()

In [ ]:
# save dataframe to csv
# df_target_user_comment.to_csv("data/target_user_comment.csv", index=False)

# Scrape Stories of User

## Single User

In [ ]:
# !!! insert target user here
target_user = "revivaltvid"

# get user stories from target user
stories = cl.user_stories(cl.user_id_from_username(target_user))

In [ ]:
stories_list = []

# loop through all stories
for story in tqdm(stories):
    
    # get all mentions from story
    mentions = []    
    if story.mentions:
        for mention in story.mentions:
            mentions.append(mention.user.username)
        mentions = re.sub(r'[\[\]\']', '', str(mentions))
    
    # append all data to stories_list list
    stories_list.append({
        'taken_at': story.taken_at,
        'story_id': story.pk,
        'code': story.code,
        'user_id': story.user.pk,
        'username': story.user.username,
        'media_type': story.media_type,
        'product_type': story.product_type,
        'sponsor_tags': story.sponsor_tags,
        'hashtags': story.hashtags,
        'mentions': mentions,
        'locations': story.locations,
        'thumbnail_url': story.thumbnail_url,
    })

# convert list to dataframe
df_stories = pd.DataFrame(stories_list)
df_stories

media_type= 1: Image, 2: Video

## Multiple Users

In [ ]:
# read target users from csv
target_users = pd.read_csv("data/users.csv")["users"].tolist()

In [ ]:
stories_list = []

# loop through all target users
for target_user in tqdm(target_users):
    # get user stories from target user
    stories = cl.user_stories(cl.user_id_from_username(target_user))

    # loop through all stories
    for story in tqdm(stories):
        
        # get all mentions from story
        mentions = []    
        if story.mentions:
            for mention in story.mentions:
                mentions.append(mention.user.username)
            mentions = re.sub(r'[\[\]\']', '', str(mentions))
        
        # append all data to stories_list list
        stories_list.append({
            'taken_at': story.taken_at,
            'story_id': story.pk,
            'code': story.code,
            'user_id': story.user.pk,
            'username': story.user.username,
            'media_type': story.media_type,
            'product_type': story.product_type,
            'sponsor_tags': story.sponsor_tags,
            'hashtags': story.hashtags,
            'mentions': mentions,
            'locations': story.locations,
            'thumbnail_url': story.thumbnail_url,
        })

In [ ]:
# convert list to dataframe
df_story = pd.DataFrame(stories_list)
df_story.head()

# Hide Me

In [ ]:
# # get all media from user by pagination (bisa delay)
    # end_cursor = None
    # user_media = []
    # per_page = 30
    # target_scrape = int(600/per_page)

    # # loop through all media
    # for media in range(target_scrape):
    #     print("Media ke-",media)
        
    #     # get media by pagination
    #     medias, end_cursor = cl.user_medias_paginated(user_id, per_page, end_cursor=end_cursor)
        
    #     # append all media to user_media
    #     for media in medias:
    #         user_media.append(media)
            
    #     # delay 5 seconds
    #     print("sleeping for 30 seconds")
    #     time.sleep(30)

In [ ]:
# # coba-coba mencari interval dan konversi zona waktu
# datetime_list = []

# for media in user_medias:
#     datetime_list.append(media.taken_at)
    
# latest = max(datetime_list)
# latest_indo = latest.astimezone(timezone_indo)
# index = datetime_list.index(latest)

# # trying to get interval
# now = datetime.now(timezone_indo)
# interval = (now - latest_indo).seconds // 3600

# print("latest")
# print(index)
# print(latest)
# print(latest_indo)

# print("now")
# print(now)

# print("interval")
# print(interval)